# Processo de ETL
Preparação da *__base de dados__*, tranformando os dados crus no formato pronto para ser aplicado nas análise.

### Importação das Bibliotecas

In [ ]:
import csv
import pandas as pd

### Carregando a base de dados original

In [ ]:
original_df = pd.read_csv('./landing/mensario.csv')

#### Remoção de campos não utilizados
Remoção: Tipo da instalação, Terminal, Tipo de operação, TEUs, Unidades

In [ ]:
original_df.columns

Index(['Ano', 'Mês', 'Berço', 'Tipo da instalação', 'Terminal',
       'Perfil da carga', 'Tipo de operação', 'Navegação', 'Sentido da carga',
       'Carga', 'Toneladas', 'TEUs', 'Unidades'],
      dtype='object')

In [ ]:
colunas_remove = ['Tipo da instalação', 'Terminal', 'Tipo de operação', 'TEUs', 'Unidades']
tratando_df = original_df.drop(colunas_remove, axis=1)

In [ ]:
#Apagando original_df não mais necessário
del original_df
#Renomeando colunas
tratando_df = tratando_df.rename(columns={'Ano': 'ano', 'Mês':'mes', 'Berço':'berco', 'Perfil da carga':'perfil', 'Navegação':'navegacao', 'Sentido da carga':'sentido', 'Carga':'carga', 'Toneladas':'toneladas'})

## Seleção do Período
Limitando o período para seleção dos dados da base

In [ ]:
#Selecionar o ano de início
inicio = 2005
#Criação do filtro
filtro = (tratando_df['ano'] >= inicio)
#Aplicação do filtro
porto_df = tratando_df[filtro]
#Ordenação da base
porto_df = porto_df.sort_values(['ano', 'mes', 'perfil', 'navegacao', 'sentido', 'carga', 'berco'])
# Remoção do DF em tratamento
del tratando_df

### Agregando os registros e seus valores
Registros de mesmas características mas com complementação de valores de toneladas movimentadas

In [ ]:
porto_df = porto_df.groupby(['ano', 'mes', 'perfil', 'navegacao', 'sentido', 'carga', 'berco']).sum().reset_index()

### Convertendo valores das toneladas
Realizando o arredondamento dos valores das toneladas para posterior conversão em inteiro, evitando assim uma maior perda de precisão com o trucamento dos valores.

In [ ]:
porto_df['toneladas'] = porto_df['toneladas'].round(0)
porto_df['toneladas'] = porto_df['toneladas'].astype('int')

# Salvando arquivo a partir porto_df
Arquivo com base no dataframe porto_df com os campos categóricos e no período 2005-2023

In [ ]:
porto_df.to_csv('./staging/porto.csv', index=False)

## Criando dataframe exclusivo da Movimentação

#### Filtrando período

In [ ]:
# Definir o período desejado
inicio = 2005
final = 2022

# Filtrar os dados pelo período desejado
periodo_df = porto_df[(porto_df['ano'] >= inicio) & (porto_df['ano'] <= final)]
# Agrupar os dados por mês e ano e calcular a soma das toneladas
movimento_df = periodo_df.groupby(['ano', 'mes'])['toneladas'].sum().reset_index()
movimento_df = movimento_df.rename(columns={'toneladas': 'porto'})

###### Dicionário de cargas

In [ ]:
carga_dict = {
    'CARNES' : 'carnes',
    'OUTRAS MERCADORIAS' : 'outras',
    'PRODUTOS QUÍMICOS' : 'prod_quimicos',
    'SAL' : 'sal',
    'SEM CARGAS' : 'sem_cargas',
    'VEÍCULOS' : 'veiculos',
    'CELULOSE' : 'celulose',
    'GRÃOS E FARELO' : 'graos',
    'PETRÓLEO E DERIVADOS' : 'petroleo',
    'PRODUTOS DA INDÚSTRIA QUÍMICA' : 'ind_quimica',
    'AÇÚCAR' : 'sugar',
    'CARVÃO' : 'carvao',
    'COMBUSTÍVEIS, ÓLEOS E PRODUTOS M' : 'combustiveis',
    'ETANOL' : 'etanol',
    'FERTILIZANTES' : 'fertiliz',
    'GÁS DE PETRÓLEO' : 'gas',
    'MINÉRIO DE FERRO' : 'ferro',
    'SUCOS' : 'sucos',
    'ÓLEO DE ORIGEM VEGETAL' : 'ol_beg',
    'PARTES/MOTORES/GERADORES/PÁS EÓL': 'mot_ger',
    'BUNKER (DIESEL)' : 'bunk_dies',
    'BUNKER (O.COMBUSTIVEL)': 'bunk_comb',
    'ENXOFRE' : 'enxofre',
    'BIODIESEL' : 'biodiesel'
}

#### Filtrando as cargas

In [ ]:
# Valor específico para o campo "carga" : GRÃOS E FARELO , AÇÚCAR , OUTRAS MERCADORIAS
valor_carga = ['GRÃOS E FARELO', 'OUTRAS MERCADORIAS', 'AÇÚCAR']
for k in valor_carga:
    carga_df = periodo_df.loc[porto_df['carga'] == k]

    # Agrupar os dados por mês e ano e calcular a soma das toneladas
    carga_df = carga_df.groupby(['ano', 'mes'])['toneladas'].sum().reset_index()
    carga_df = carga_df.rename(columns={'toneladas': carga_dict[k]})

    # Adicionar a coluna do segundo DataFrame ao primeiro DataFrame usando a coluna em comum
    movimento_df = movimento_df.join(carga_df[carga_dict[k]])

## Salvando arquivo a partir movimento_df
Arquivo com base no dataframe movimento_df com os campos de movimentação das cargas selecionadas bem como o período 2005-2023

In [ ]:
movimento_df.to_csv('./staging/porto_ts.csv', index=False)

In [ ]:
movimento_df

,ano,mes,porto,graos,outras,sugar
0,2005,1,5027941,412400,1780376,581383
1,2005,2,5316605,707274,1821686,878016
2,2005,3,5899912,1221079,1917759,610140
3,2005,4,5850639,1301598,1976311,764285
4,2005,5,6826372,1575474,2122232,1007705
...,...,...,...,...,...,...
211,2022,8,14626790,4122517,3188229,2455597
212,2022,9,13628567,3743920,3258989,2079487
213,2022,10,14435751,4185655,3198681,2249483
214,2022,11,12194409,3459541,2558110,2199680


### Base real posterior

In [ ]:
posterior_df = pd.read_csv('./landing/mensario_2023.csv')

In [ ]:
colunas_remove = ['TIPO_INSTALACAO', 'TERMINAL',
                  'TIPO_MOVIMENTO', 'TEUS', 'UNIDADES']
posterior_df = posterior_df.drop(colunas_remove, axis=1)

In [ ]:
# Selecionar o ano de início
inicio = 2005
# Criação do filtro
filtro = (posterior_df['ANO'] >= inicio)
# Aplicação do filtro
posterior_df = posterior_df[filtro]
# Ordenação da base
posterior_df = posterior_df.sort_values(
    ['ANO', 'MES', 'BERCO', 'PERFIL_CARGA', 'NAVEGACAO', 'SENTIDO_CARGA', 'CARGA'])
posterior_df = posterior_df.rename(columns={'ANO': 'ano', 'MES': 'mes', 'BERCO': 'berco', 'PERFIL_CARGA': 'perfil',
                                   'NAVEGACAO': 'navegacao', 'SENTIDO_CARGA': 'sentido', 'CARGA': 'carga', 'TONELADAS': 'toneladas'})
posterior_df = posterior_df.groupby(
    ['ano', 'mes', 'perfil', 'navegacao', 'sentido', 'carga', 'berco']).sum().reset_index()
posterior_df['toneladas'] = posterior_df['toneladas'].round(0)
posterior_df['toneladas'] = posterior_df['toneladas'].astype('int')

# Definir o período desejado
inicio = 2023
final = 2023

# Definir quais as Cargas desejadas
filtro_carga = ('TODAS', 'GRÃOS E FARELO', 'OUTRAS MERCADORIAS', 'AÇÚCAR')

# Filtrar os dados pelo período desejado
posterior_df = posterior_df[(posterior_df['ano'] >= inicio) & (
    posterior_df['ano'] <= final)]

# Agrupar os dados por mês e ano e calcular a soma das toneladas
posterior_df = posterior_df.groupby(
    ['ano', 'mes'])['toneladas'].sum().reset_index()
posterior_df = posterior_df.rename(columns={'toneladas': 'porto'})

In [ ]:
# Conversão de ano/mes em index
posterior_df['data'] = pd.to_datetime(posterior_df['ano'].astype(
    str) + '-' + posterior_df['mes'].astype(str), format='%Y-%m')
posterior_df = posterior_df.drop(['ano', 'mes'], axis=1)
data = posterior_df.pop('data')  # Remove a coluna e armazena em uma variável
posterior_df.insert(0, 'data', data)  # Insere a coluna na posição desejada
del data
posterior_df['data'] = pd.to_datetime(posterior_df['data'])

posterior_df

,data,porto
0,2023-01-01,10116563
1,2023-02-01,10859885
2,2023-03-01,15173703
3,2023-04-01,13582911
4,2023-05-01,14997681
5,2023-06-01,15458725
6,2023-07-01,15191901


## Salvando arquivo a partir movimento_df
Arquivo com base no dataframe movimento_df com os campos de movimentação das cargas selecionadas bem como o período 2005-2023

In [ ]:
posterior_df.to_csv('./staging/posterior_ts.csv', index=False)